In [1]:
import itertools
!pip install pymorphy2
from pymorphy2 import MorphAnalyzer
from collections import Counter, defaultdict
import numpy as np
from string import punctuation
from nltk.tokenize import sent_tokenize
morph = MorphAnalyzer()

     |████████████████████████████████| 51kB 1.4MB/s 
     |████████████████████████████████| 7.1MB 7.7MB/s 


# Задание 1.

In [2]:
!wget https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/besy_dostoevsky.txt
!wget https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/anna_karenina.txt
tolstoy = open('anna_karenina.txt').read()
dostoevsky = open('besy_dostoevsky.txt').read()

--2019-10-23 20:47:27--  https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/besy_dostoevsky.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2334469 (2.2M) [text/plain]
Saving to: ‘besy_dostoevsky.txt’

besy_dostoevsky.txt 100%[===================>]   2.23M  --.-KB/s    in 0.1s    

2019-10-23 20:47:28 (21.3 MB/s) - ‘besy_dostoevsky.txt’ saved [2334469/2334469]

--2019-10-23 20:47:30--  https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/anna_karenina.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting res

In [0]:
def normalize(text):
    normalized_text = [morph.parse(word.strip(punctuation))[0].normal_form for word in text.lower().split()]
    normalized_text = [word for word in normalized_text if word]
    return normalized_text

norm_tolstoy = normalize(tolstoy)
norm_dostoevsky = normalize(dostoevsky)

In [31]:
import nltk
nltk.download('punkt')
sentences_tolstoy = [['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(tolstoy)]
sentences_tolstoy = sentences_tolstoy[:1000]
sentences_dostoevsky = [['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(dostoevsky)]
sentences_dostoevsky = sentences_dostoevsky[:1000]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
def ngrammer(tokens, n):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

Биграм

In [0]:
unigrams_tolstoy = Counter()
bigrams_tolstoy = Counter()

for sentence in sentences_tolstoy:
	unigrams_tolstoy.update(sentence)
	bigrams_tolstoy.update(ngrammer(sentence, 2))

unigrams_dostoevsky = Counter()
bigrams_dostoevsky = Counter()

for sentence in sentences_dostoevsky:
	unigrams_dostoevsky.update(sentence)
	bigrams_dostoevsky.update(ngrammer(sentence, 2))

матрикс

In [0]:
matrix_dostoevsky = np.zeros((len(unigrams_dostoevsky), len(unigrams_dostoevsky))) # 初始值0矩阵
id2word_dostoevsky = list(unigrams_dostoevsky) # 列表值为('word', count)
word2id_dostoevsky = {word:i for i, word in enumerate(id2word_dostoevsky)} # 字典值为('word', count):0（矩阵行和列定位）

for ngram in bigrams_dostoevsky: # 矩阵传值
    word1, word2 = ngram.split()
    matrix_dostoevsky[word2id_dostoevsky[word1]][word2id_dostoevsky[word2]] =  (bigrams_dostoevsky[ngram]/unigrams_dostoevsky[word1])

matrix_tolstoy = np.zeros((len(unigrams_tolstoy), len(unigrams_tolstoy)))

id2word_tolstoy = list(unigrams_tolstoy)
word2id_tolstoy = {word:i for i, word in enumerate(id2word_tolstoy)}

for ngram in bigrams_tolstoy:
    word1, word2 = ngram.split()
    matrix_tolstoy[word2id_tolstoy[word1]][word2id_tolstoy[word2]] =  (bigrams_tolstoy[ngram]/unigrams_tolstoy[word1])

генерация

In [38]:
def generate(matrix, id2word, word2id, n=100, start='<start>'):
    text = []
    current_idx = word2id[start]
    
    for i in range(n):
        
        chosen = np.random.choice(matrix.shape[1], p=matrix[current_idx])
        text.append(id2word[chosen])
        
        if id2word[chosen] == '<end>':
            chosen = word2id['<start>']
        current_idx = chosen
    
    return ' '.join(text)
print(generate(matrix_dostoevsky, id2word_dostoevsky, word2id_dostoevsky).replace('<end>', '\n'))
print(generate(matrix_tolstoy, id2word_tolstoy, word2id_tolstoy).replace('<end>', '\n'))

варвар пётр 
 кстати уж конечно никто и так много о портрет поэт воплотить укоризна сохраняться и не приметить до тот подписаться 
 — и мы столько год в скобка что ещё не в гнев 
 впрочем нет это смеяться 
 — он быть возможно быть хотя в шестьдесят третье час только голова в то есть и мы стать плоть от весь это не слышать что-то страшный быть и у она и стоять уставать и даровитый человек хоть ждать ли вы свой семилетний и счесть себя дробный сам забытый и эксплуатировать труд материал собрать и о право и мы —
оказаться что быть ребёнок когда левин войти старый гувернантка о тот боль в зоологический сад обвиснуть весь он ты другой в рука стоялый разговаривать с развратный отец удержать лёгкий туман который он 
 в дом щербацкий делать но надо же вы я заплатить 
 – ну в тот дело быть бы позвать ты и англичанин выработать бы калач с она вскоре после тот что даже этот молитва который она производить в рука улыбаться 
 – продолжать – закричать она несколько раз как это быть 
 степан аркадьй 

триграмм

In [0]:
sentences_tolstoy_2 = [['<start>'] + ['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(tolstoy)]
sentences_tolstoy_2 = sentences_tolstoy_2[:1000]
sentences_dostoevsky_2 = [['<start>'] + ['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(dostoevsky)]
sentences_dostoevsky_2 = sentences_dostoevsky_2[:1000]

In [0]:
unigrams_tolstoy_2 = Counter()
bigrams_tolstoy_2 = Counter()
trigrams_tolstoy_2 = Counter()

for sentence in sentences_tolstoy_2:
  unigrams_tolstoy_2.update(sentence)
  bigrams_tolstoy_2.update(ngrammer(sentence, 2))
  trigrams_tolstoy.update(ngrammer(sentence, 3))

unigrams_dostoevsky_2 = Counter()
bigrams_dostoevsky_2 = Counter()
trigrams_dostoevsky_2 = Counter()

for sentence in sentences_dostoevsky_2:
  unigrams_dostoevsky_2.update(sentence)
  bigrams_dostoevsky_2.update(ngrammer(sentence, 2))
  trigrams_dostoevsky_2.update(ngrammer(sentence, 3))

матрикс

In [0]:
matrix_dostoevsky_2 = np.zeros((len(bigrams_dostoevsky_2), len(unigrams_dostoevsky_2))) # 初始值0矩阵
id2word_dostoevsky_u = list(unigrams_dostoevsky_2) # 列表值为('word', count)
word2id_dostoevsky_u = {word:i for i, word in enumerate(id2word_dostoevsky_u)} # 字典值为('word', count):0（矩阵列定位）
id2word_dostoevsky_b = list(bigrams_dostoevsky_2) # 列表值为('word', count)
word2id_dostoevsky_b = {word:i for i, word in enumerate(id2word_dostoevsky_b)} # 字典值为('word', count):0（矩阵行定位）

for ngram in trigrams_dostoevsky_2: # 矩阵传值
  word1, word2, word3 = ngram.split()
  bigram = "{} {}".format(word1, word2)
  matrix_dostoevsky_2[word2id_dostoevsky_b[bigram]][word2id_dostoevsky_u[word3]] =  (trigrams_dostoevsky_2[ngram]/bigrams_dostoevsky_2[bigram])

In [0]:
matrix_tolstoy_2 = np.zeros((len(bigrams_tolstoy_2), len(unigrams_tolstoy_2)))
id2word_tolstoy_u = list(unigrams_tolstoy_2) # 列表值为('word', count)
word2id_tolstoy_u = {word:i for i, word in enumerate(id2word_tolstoy_u)} # 字典值为('word', count):0（矩阵列定位）
id2word_tolstoy_b = list(bigrams_tolstoy_2) # 列表值为('word', count)
word2id_tolstoy_b = {word:i for i, word in enumerate(id2word_tolstoy_b)} # 字典值为('word', count):0（矩阵行定位）

for ngram in trigrams_tolstoy_2:
    word1, word2, word3 = ngram.split()
    bigram = "{} {}".format(word1, word2)
    matrix_tolstoy[word2id_tolstoy_b[bigram]][word2id_tolstoy_u[word3]] =  (trigrams_tolstoy[ngram]/bigrams_tolstoy[bigram])

генерация

In [64]:
def generate_2(matrix, id2word_u, word2id_u, id2word_b, word2id_b, start = '<start> <start>', n=100):
    text = []
    current_idx = word2id_b[start]
    
    for i in range(n):
      chosen = np.random.choice(matrix.shape[1], p=matrix[current_idx])
      text.append(id2word_u[chosen])
        
      if id2word_u[chosen] == '<start>' and start.split()[1] != '<start>':
        id2word_u[chosen] = '<end>'
        
      if id2word_u[chosen] == '<end>':
        chosen = word2id_b['<start> <start>']
        
      start = start.split()[1] + ' ' + id2word_u[chosen]
          
      current_idx = word2id_b[start]
    
    return ' '.join(text)
print(generate_2(matrix_dostoevsky_2, id2word_dostoevsky_u, word2id_dostoevsky_u, id2word_dostoevsky_b, word2id_dostoevsky_b).replace('<end>', '\n'))
print(generate_2(matrix_tolstoy_2, id2word_tolstoy_u, word2id_tolstoy_u, id2word_tolstoy_b, word2id_tolstoy_b).replace('<end>', '\n'))

KeyError: ignored

# Задание 2.

In [0]:
import re, gensim
from gensim.utils import tokenize
from gensim.summarization.textcleaner import split_sentences

In [0]:
text = '''Президент США сообщил, что правительство Турции проинформировало о прекращении боевых действий. По его мнению, перемирие на севере Сирии будет соблюдаться постоянно.

Господин Трамп назвал заслугой США перемирие, благодаря которому курды смогли спокойно покинуть границу с Турцией. «Этого добились мы, Соединенные Штаты, а не кто-то еще, никакая другая страна»,— сказал глава государства. По его словам, США удалось сохранить жизни многих курдов.

Дональд Трамп сообщил, что на территории Сирии останется небольшое число военных — «в районах, где есть нефть». «Мы будем ее защищать и решим, что делать с этим в будущем»,— сказал президент США.

Турция начала военную операцию «Источник мира» в Сирии 9 октября. В полночь 18 октября Анкара приостановила операцию. Ее завершили сегодня, 23 октября. Однако Турция не намерена полностью уходить с территории Сирии. В рамках договоренностей Вашингтон пообещал вывести курдские формирования за демаркационную линию в 30 км к югу от границы Сирии с Турцией. Россия и Турция начнут совместное патрулирование освобожденной курдами территории.
'''

In [46]:
sents_text = list(split_sentences(text))
print(sents_text)

['Президент США сообщил, что правительство Турции проинформировало о прекращении боевых действий.', 'По его мнению, перемирие на севере Сирии будет соблюдаться постоянно.', 'Господин Трамп назвал заслугой США перемирие, благодаря которому курды смогли спокойно покинуть границу с Турцией.', '«Этого добились мы, Соединенные Штаты, а не кто-то еще, никакая другая страна»,— сказал глава государства.', 'По его словам, США удалось сохранить жизни многих курдов.', 'Дональд Трамп сообщил, что на территории Сирии останется небольшое число военных — «в районах, где есть нефть».', '«Мы будем ее защищать и решим, что делать с этим в будущем»,— сказал президент США.', 'Турция начала военную операцию «Источник мира» в Сирии 9 октября.', 'В полночь 18 октября Анкара приостановила операцию.', 'Ее завершили сегодня, 23 октября.', 'Однако Турция не намерена полностью уходить с территории Сирии.', 'В рамках договоренностей Вашингтон пообещал вывести курдские формирования за демаркационную линию в 30 км к

In [77]:
symbols_sents = []
for sent in sents_text:
  symbols = []
  for smbl in sent:
    symbols.append(smbl)
  symbols_sents.append(symbols)
print(symbols_sents)

[['П', 'р', 'е', 'з', 'и', 'д', 'е', 'н', 'т', ' ', 'С', 'Ш', 'А', ' ', 'с', 'о', 'о', 'б', 'щ', 'и', 'л', ',', ' ', 'ч', 'т', 'о', ' ', 'п', 'р', 'а', 'в', 'и', 'т', 'е', 'л', 'ь', 'с', 'т', 'в', 'о', ' ', 'Т', 'у', 'р', 'ц', 'и', 'и', ' ', 'п', 'р', 'о', 'и', 'н', 'ф', 'о', 'р', 'м', 'и', 'р', 'о', 'в', 'а', 'л', 'о', ' ', 'о', ' ', 'п', 'р', 'е', 'к', 'р', 'а', 'щ', 'е', 'н', 'и', 'и', ' ', 'б', 'о', 'е', 'в', 'ы', 'х', ' ', 'д', 'е', 'й', 'с', 'т', 'в', 'и', 'й', '.'], ['П', 'о', ' ', 'е', 'г', 'о', ' ', 'м', 'н', 'е', 'н', 'и', 'ю', ',', ' ', 'п', 'е', 'р', 'е', 'м', 'и', 'р', 'и', 'е', ' ', 'н', 'а', ' ', 'с', 'е', 'в', 'е', 'р', 'е', ' ', 'С', 'и', 'р', 'и', 'и', ' ', 'б', 'у', 'д', 'е', 'т', ' ', 'с', 'о', 'б', 'л', 'ю', 'д', 'а', 'т', 'ь', 'с', 'я', ' ', 'п', 'о', 'с', 'т', 'о', 'я', 'н', 'н', 'о', '.'], ['Г', 'о', 'с', 'п', 'о', 'д', 'и', 'н', ' ', 'Т', 'р', 'а', 'м', 'п', ' ', 'н', 'а', 'з', 'в', 'а', 'л', ' ', 'з', 'а', 'с', 'л', 'у', 'г', 'о', 'й', ' ', 'С', 'Ш', 'А', ' ',

In [0]:
p = gensim.models.Phrases(symbols_sents, scoring='npmi', threshold=-1)

In [79]:
p1 = gensim.models.Phrases(p[symbols_sents], scoring='npmi', threshold=-1)

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [80]:
p2 = gensim.models.Phrases(p1[p[symbols_sents]], scoring='npmi', threshold=-1)

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [81]:
p3 = gensim.models.Phrases(p2[p1[p[symbols_sents]]], scoring='npmi', threshold=-1)

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [82]:
list(p3[p2[p1[p[symbols_sents]]]])

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


[['П',
  'р_е',
  'з',
  'и',
  'д_е',
  'н',
  'т_ ',
  'С',
  'Ш',
  'А',
  ' _с',
  'о',
  'о_б',
  'щ',
  'и_л',
  ',_ ',
  'ч',
  'т_о',
  ' _п',
  'р_а',
  'в',
  'и_т',
  'е',
  'л',
  'ь',
  'с_т',
  'в_о',
  ' _Т',
  'у_р',
  'ц_и',
  'и_ ',
  'п_р',
  'о',
  'и_н',
  'ф',
  'о_р',
  'м_и',
  'р_о',
  'в_а',
  'л',
  'о_ ',
  'о_ ',
  'п_р',
  'е',
  'к',
  'р_а',
  'щ',
  'е_н',
  'и_и',
  ' _б',
  'о_е',
  'в',
  'ы',
  'х',
  ' _д',
  'е',
  'й',
  'с_т',
  'в',
  'и',
  'й',
  '.'],
 ['П',
  'о_ ',
  'е',
  'г_о',
  ' ',
  'м',
  'н_е',
  'н_и',
  'ю',
  ',_ ',
  'п',
  'е_р',
  'е_м',
  'и_р',
  'и_е',
  ' _н',
  'а_ ',
  'с',
  'е',
  'в',
  'е_р',
  'е_ ',
  'С_и',
  'р_и',
  'и_ ',
  'б',
  'у_д',
  'е',
  'т_ ',
  'с_о',
  'б',
  'л',
  'ю',
  'д_а',
  'т_ь',
  'с',
  'я_ ',
  'п_о',
  'с_т',
  'о',
  'я',
  'н_н',
  'о',
  '.'],
 ['Г',
  'о_с',
  'п_о',
  'д',
  'и_н',
  ' _Т',
  'р_а',
  'м',
  'п',
  ' _н',
  'а',
  'з',
  'в_а',
  'л',
  ' ',
  'з_а',
  'с',
  'л'